In [10]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os.path
import pathpy as pp
import my_functions
from my_functions import matprint
import igraph
import csv

from IPython.display import * # no idea what this does
from IPython.display import HTML # no idea what this does

**csh tutorial:** https://ingoscholtes.github.io/pathpy/tutorial.html#data

# 1. Flights
US California flights in quarter 1 of 2011, take a subset for analyses.

**flight data:** https://www.transtats.bts.gov/Tables.asp?QO_VQ=EED&QO_anzr=Nv4%FDPn44vr4%FDf6n6v56vp5%FD%FLS14z%FDHE%FDg4nssvp%FM-%FD%FDh.f.%FDPn44vr45&QO_fu146_anzr=Nv4%FDPn44vr45 (T-100 Domestic Segment (U.S. Carriers)

**origin-destination data:** https://www.transtats.bts.gov/Tables.asp?QO_VQ=EFI&QO_anzr=Nv4yv0r%FDb4vtv0%FDn0q%FDQr56v0n6v10%FDf748rB%FD%FLQOEO%FM&QO_fu146_anzr=b4vtv0%FDn0q%FDQr56v0n6v10%FDf748rB (DB1B Coupon)

## 1(a)Process origin-destination data

In [11]:
PATH = "Data/US flights 2011/od_stats.csv"

od_df = pd.read_csv(PATH)
od_df = od_df[["ITIN_ID","SEQ_NUM","COUPONS","ORIGIN","DEST","PASSENGERS"]]
od_df =  od_df.sort_values(by=['ITIN_ID','SEQ_NUM'])
#od_df = od_df.iloc[:2000,:] # sample first 2000 to make stuff quicker first
#od_df.columns = ["source","target","frequency"]
od_df.head()

,ITIN_ID,SEQ_NUM,COUPONS,ORIGIN,DEST,PASSENGERS
1,2011128,4,7,PHX,SJC,1.0
0,2011128,5,7,SJC,PHX,1.0
2,20111103,2,4,CLT,LAX,1.0
3,20111103,3,4,LAX,PHL,1.0
4,20111147,3,3,PHX,ONT,1.0


In [12]:
paths = od_df[["ITIN_ID","ORIGIN","DEST","PASSENGERS"]].groupby(['ITIN_ID',"PASSENGERS"],as_index=False).agg({'ORIGIN':lambda x: ','.join(x),
                                                                                    "DEST":lambda x: ','.join(x)})
paths['DEST'] = paths['DEST'].apply(lambda x: x[-3:])
paths['PATH'] = paths['ORIGIN']+','+paths['DEST']
paths['LENGTH'] = paths['PATH'].apply(lambda x: x.count(',')+1)
paths = paths.groupby(['PATH','LENGTH'],as_index=False).agg({"PASSENGERS": lambda x: sum(x)})
paths.head()

,PATH,LENGTH,PASSENGERS
0,"ABQ,BUR",2,16.0
1,"ABQ,BUR,ABQ",3,9.0
2,"ABQ,BUR,DFW",3,1.0
3,"ABQ,BUR,LAS",3,5.0
4,"ABQ,BUR,LAX,ABQ",4,1.0


In [13]:
# make smaller for now but change later
print(paths.shape)
paths = paths.sample(frac=0.01)
print(paths.shape)

(35801, 3)
(358, 3)


In [14]:
# make way smaller for now
paths

to_ngram = paths["PATH"]+','+paths["PASSENGERS"].astype(str)
to_ngram.to_csv("Data/US flights 2011/US flights od.ngram",
                                    sep='\t',
                                    index=False,
                                    header=False)

# printing weird stuff at the end I need to fix this

## 1(b) Data from departures dataset

In [5]:
PATH = "Data/US flights 2011/US flights 2011.csv"
dep_df = pd.read_csv(PATH)
dep_df = dep_df[dep_df.DEPARTURES_PERFORMED>0]
dep_df = dep_df[dep_df.QUARTER==1]
#dep_df = dep_df.sample(frac=0.1)
dep_df = dep_df[["ORIGIN","DEST"]]
dep_df.columns = ["source","target"]
dep_df.to_csv("Data/US flights 2011/US flights 2011.edges",index=False)
dep_df.head()

,source,target
25,IFP,SMF
26,LAX,RNO
29,ONT,AZA
30,LAX,CLE
31,BFI,SJC


# San Francisco Taxi Data